In [6]:
def parse_input_file(filename):
    with open(filename, "r") as file:
        lines = file.readlines()
    
    # Parse first line (D, R, T)
    D, R, T = map(int, lines[0].split())
    
    # Parse resources
    resources = []
    for i in range(1, R + 1):
        parts = lines[i].split()
        resource = {
            "Resource ID": int(parts[0]), 
            "Activation Cost": int(parts[1]),  # One-time initial expenditure
            "Periodic Cost": int(parts[2]),    # Recurring maintenance cost per turn
            "Active Turns": int(parts[3]),     # Turns the resource stays active
            "Downtime Turns": int(parts[4]),   # Turns needed for maintenance after a cycle
            "Life Cycle": int(parts[5]),       # Total lifespan of the resource
            "Buildings Powered": int(parts[6]),# Number of buildings it supports per active turn
            "Special Effect": parts[7],        # Unique effect or property
            "Efficiency Rating": int(parts[8]) if len(parts) > 8 else None  # Additional performance metric
        }
        resources.append(resource)
    
    # Parse turns
    turns = []
    for i in range(R + 1, R + 1 + T):
        TMt, TXt, TRt = map(int, lines[i].split())
        turns.append({"Minimum Buildings": TMt, "Maximum Buildings": TXt, "Profit": TRt})
    
    return D, R, T, resources, turns

level = "1-thunberg"

D, R, T, resources, turns = parse_input_file(f"{level}.txt")

# Print parsed data
print("Initial Capital:", D)
print("Total Resources:", R)
print("Game Turns:", T)
print("Resources:", resources)
print("Turns:", turns)



Initial Capital: 1000
Total Resources: 15
Game Turns: 50
Resources: [{'Resource ID': 0, 'Activation Cost': 533, 'Periodic Cost': 12, 'Active Turns': 1, 'Downtime Turns': 1, 'Life Cycle': 4, 'Buildings Powered': 2, 'Special Effect': 'X', 'Efficiency Rating': None}, {'Resource ID': 1, 'Activation Cost': 12, 'Periodic Cost': 7, 'Active Turns': 3, 'Downtime Turns': 2, 'Life Cycle': 10, 'Buildings Powered': 1, 'Special Effect': 'A', 'Efficiency Rating': 2}, {'Resource ID': 2, 'Activation Cost': 89, 'Periodic Cost': 36, 'Active Turns': 1, 'Downtime Turns': 0, 'Life Cycle': 2, 'Buildings Powered': 0, 'Special Effect': 'E', 'Efficiency Rating': 11}, {'Resource ID': 3, 'Activation Cost': 238, 'Periodic Cost': 49, 'Active Turns': 3, 'Downtime Turns': 1, 'Life Cycle': 6, 'Buildings Powered': 0, 'Special Effect': 'D', 'Efficiency Rating': 10}, {'Resource ID': 4, 'Activation Cost': 36, 'Periodic Cost': 5, 'Active Turns': 1, 'Downtime Turns': 0, 'Life Cycle': 6, 'Buildings Powered': 1, 'Special Effe

In [ ]:
## GAME SEQUENCE

available_resources = resources.copy()
available_budget = D
active_resources = []

for turn in range(T):
    

    resources_to_activate = []
    
    
    ##TODO IMPLEMENT THE special effects of everything
    ## Assert whether at any turn we spent more than we have 
    for resource in resources_to_activate:
        available_budget -= resource["Activation Cost"]
        if available_budget < 0:
            print("Not enough budget")
            break
    for resource in active_resources:
        available_budget -= resource["Periodic Cost"]
        if available_budget < 0:
            print("Not enough budget")
            break
    
    
    ## add the resources to the active resources
    for resource in resources_to_activate:
        active_resources.append(resource)
        # available_resources.remove(resource)
    # print whether we have more building that we would obtain profit from or if there aren't enough buildings powered
    if sum([resource["Buildings Powered"] for resource in active_resources]) > turns[turn]["Maximum Buildings"]:
            print("Too many buildings powered")

    if sum([resource["Buildings Powered"] for resource in active_resources]) < turns[turn]["Minimum Buildings"]:
        profit = 0
        print("Not enough buildings powered")
    else :
        #Calculate profit for each turn and then add it to the available budget
        profit = turns[turn]["Profit"] * min(sum([resource["Buildings Powered"] for resource in active_resources]), turns[turn]["Maximum Buildings"])
        available_budget += profit

    ## Check inside the active resources if any of them expired
    for resource in active_resources:
        resource["Active Turns"] -= 1
        if resource["Active Turns"] == 0:
            active_resources.remove(resource)

    # print the output in the format requested
    print(f"{turn +1} {len(resources_to_activate)} {[resource['Resource ID'] for resource in active_resources]}")
